In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime

%load_ext autoreload
%autoreload 2
import resources

# import warnings
# warnings.filterwarnings(action= 'ignore')

In [2]:
# utilizamos la funcion readAllSheets para leer todas las hojas de nuestro archivo xlxs, y guardamos sus datos en un diccionario y sus nombres (de las hojas) en una lista

dfs, names = resources.readAllSheets('../materiales/homicidios.xlsx')

In [3]:
names

['HECHOS', 'DICCIONARIO_HECHOS', 'VICTIMAS', 'DICCIONARIO_VICTIMAS', 'clas']

In [4]:
# Guardamos el primer dataset 'HECHOS' en una variable

df = dfs[names[0]]

In [5]:
df.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


In [6]:
# Vemos las columnas de nuestro dataframe

df.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [7]:
# Tomando en cuenta la definicion de cada columna o feature desarrollada en el diccionario de datos, 
# seleccionamos las columnas que nos interesan y que cuentan con datos relevantes para el analisis, desechamos columnas no relevantes para el analisis o que contienen datos que ya tenemos o redundantes

df_homicidios = df[['ID','N_VICTIMAS','FECHA','HH','LUGAR_DEL_HECHO','TIPO_DE_CALLE','COMUNA','pos x','pos y','PARTICIPANTES','VICTIMA', 'ACUSADO']]

In [8]:
# Vemos los tipos de datos de las columnas

df_homicidios.dtypes

ID                         object
N_VICTIMAS                  int64
FECHA              datetime64[ns]
HH                         object
LUGAR_DEL_HECHO            object
TIPO_DE_CALLE              object
COMUNA                      int64
pos x                      object
pos y                      object
PARTICIPANTES              object
VICTIMA                    object
ACUSADO                    object
dtype: object

In [9]:
# Renombramos la columna 'ID' para utilizarla como la columna de referencia para realizar un merge de dataframes

df_homicidios = df_homicidios.rename(columns= {'ID': 'ID_hecho'})

In [10]:
# Vemos un resumen general de diversos aspectos del dataframe

df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID_hecho         696 non-null    object        
 1   N_VICTIMAS       696 non-null    int64         
 2   FECHA            696 non-null    datetime64[ns]
 3   HH               696 non-null    object        
 4   LUGAR_DEL_HECHO  696 non-null    object        
 5   TIPO_DE_CALLE    696 non-null    object        
 6   COMUNA           696 non-null    int64         
 7   pos x            696 non-null    object        
 8   pos y            696 non-null    object        
 9   PARTICIPANTES    696 non-null    object        
 10  VICTIMA          696 non-null    object        
 11  ACUSADO          696 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 65.4+ KB


In [11]:
df_homicidios.duplicated().value_counts() # no existen registros duplicados

False    696
dtype: int64

In [12]:
# vemos si existen valores de 'ID_hechos' repetidos

df_homicidios['ID_hecho'].duplicated().sum()

0

In [13]:
# Vemos los valores nulos presentes en el dataframe

df_homicidios.isna().sum()

ID_hecho           0
N_VICTIMAS         0
FECHA              0
HH                 0
LUGAR_DEL_HECHO    0
TIPO_DE_CALLE      0
COMUNA             0
pos x              0
pos y              0
PARTICIPANTES      0
VICTIMA            0
ACUSADO            0
dtype: int64

In [14]:
# asignamos el segundo dataset con datos 'VICTIMAS' a un dataframe y lo guardamos en una variable

df_victimas = dfs[names[2]]

In [15]:
# vemos un resumen general de diversos aspectos del dataframe

df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [16]:
df_victimas

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,2021,12,12,PEATON,PEATON,FEMENINO,50,2021-12-12 00:00:00
713,2021-0093,2021-12-13,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
714,2021-0094,2021-12-20,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20 00:00:00
715,2021-0095,2021-12-30,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02 00:00:00


In [17]:
df_victimas.duplicated().value_counts() # no existen registros duplicados

False    717
dtype: int64

In [18]:
# vemos si existen valores nulos presentes en 'ID_hechos'

df_victimas['ID_hecho'].duplicated().sum()

21

In [19]:
# La repeticion de valores en la columna 'ID_hechos' en el dataset dataframe df_victimas, en conjunto con la diferencia de registros entre los dos datasets que manejamos,
# nos dice que existen siniestros en donde hubo mas de una victima

In [20]:
# buscamos valores nulos

df_victimas.isna().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

In [21]:
# una vez hecho el analisis de registros duplciados, vemos que existen mas registros de victimas que registros de hechos.. 
# esto puede ser porque existen mas da una victima por hecho registrado.

In [22]:
duplicados = df_victimas[df_victimas['ID_hecho'].duplicated()]
duplicados

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
30,2016-0041,2016-03-29,2016,3,29,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,SD,2016-03-30 00:00:00
99,2016-0126,2016-09-18,2016,9,18,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,60,SD
164,2017-0026,2017-02-26,2017,2,26,CONDUCTOR,AUTO,MASCULINO,19,2017-02-26 00:00:00
174,2017-0035,2017-03-23,2017,3,23,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,32,2017-03-23 00:00:00
175,2017-0035,2017-03-23,2017,3,23,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,30,2017-03-23 00:00:00
177,2017-0036,2017-03-29,2017,3,29,CONDUCTOR,MOTO,MASCULINO,20,2017-03-29 00:00:00
187,2017-0050,2017-04-28,2017,4,28,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,16,2017-04-28 00:00:00
238,2017-0108,2017-09-02,2017,9,2,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,10,2017-09-02 00:00:00
243,2017-0112,2017-09-10,2017,9,10,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,SD,2017-09-13 00:00:00
254,2017-0126,2017-10-14,2017,10,14,CONDUCTOR,AUTO,MASCULINO,36,2017-10-14 00:00:00


In [23]:
len(duplicados)

21

In [24]:
# tomemos un ejemplo de estos valores duplicados de 'ID_hecho', en base al dataframe df_victimas

df_victimas[df_victimas['ID_hecho'] == '2016-0041']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
29,2016-0041,2016-03-29,2016,3,29,CONDUCTOR,MOTO,MASCULINO,54,2016-03-29 00:00:00
30,2016-0041,2016-03-29,2016,3,29,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,SD,2016-03-30 00:00:00


In [25]:
registros_duplicados = df_victimas[df_victimas['ID_hecho'].isin(duplicados['ID_hecho'].values)]

In [26]:
len(registros_duplicados)

41

In [27]:
# aqui lo vemos mas detenidamente, no existen registros duplicados, pero existen valores de 'ID_hechos' duplicados, ya que hacen referencia al mismo hecho/siniestro
# pero haciendo referencia a distintas victimas fatales en su registro

In [28]:
# Veamos



In [29]:
# unimos ambos dataframes (df_homicidios y df_victimas)

df_unido = df_victimas.merge(df_homicidios, on= ['ID_hecho', 'FECHA'], how= 'left')


In [30]:
df_unido[df_unido['ID_hecho'] == '2017-0108']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA_x,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA_y,ACUSADO
237,2017-0108,2017-09-02,2017,9,2,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,17,2017-09-02 00:00:00,2.0,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,12.0,-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO
238,2017-0108,2017-09-02,2017,9,2,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,10,2017-09-02 00:00:00,2.0,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,12.0,-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO


In [31]:
df_unido.shape

(717, 20)

In [32]:
# desechamos columnas que cuentan con datos recursivos o que no cuentan con informacion relevante o con informacion recursiva

df_unido.drop(columns= ['AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO'], inplace= True)

In [33]:
# cambiamos el nombre de la columna VICTIMA_x a VEHICULO_FALLECIDO, ya que esta columna hace referencia al vehiculo en el que se encontraba en el momento del hecho
# la persona fallecida
# Y tambien cambiamos el nombre de la columna VICTIMA_y a VICTIMA, ya que es el que toma el papel de victimario o perjudicado en el hecho, pero no necesariamente el 
# que tiene victimas fatales o lesionados.

# En nuestro diccionario de datos de ambos datasets (hechos y victimas), se da una definicion igual para cada una de las columnas 'victima', correspondientes a cada dataset.
# Cosa que en la practica no esta del todo correcto ya que en un caso (dataset hechos) hace referencia a la posicion 'legal' de las partes en el hecho o siniestro, mientras que en
# el dataset victimas hace referencia a que posicion ocupaba el fallecido, independientemente de si se encontraba en el lugar de acusado o victima.
# Podemos decir que se existe un error interpretativo en el armado del dataset y/o recoleccion y asignacion de los datos.

In [34]:
# Renombramos las columnas correspondientes

df_unido = df_unido.rename(columns= {'VICTIMA_x': 'VEHICULO_FALLECIDO'})
df_unido = df_unido.rename(columns= {'VICTIMA_y': 'VICTIMA'})

# Renombramos tambien la columna 'HH' a 'HORA_HECHO'

df_unido = df_unido.rename(columns= {'HH': 'HORA_HECHO'})

# Renombramos pos x y pos y a LONGITUD y LATITUD respectivamente

df_unido = df_unido.rename(columns= {'pos x': 'LONGITUD'})
df_unido = df_unido.rename(columns= {'pos y': 'LATITUD'})

In [35]:
# veamos luego del merge entre dataframes si existen valores nulos

df_unido.isna().sum()

ID_hecho              0
FECHA                 0
ROL                   0
VEHICULO_FALLECIDO    0
SEXO                  0
EDAD                  0
N_VICTIMAS            2
HORA_HECHO            2
LUGAR_DEL_HECHO       2
TIPO_DE_CALLE         2
COMUNA                2
LONGITUD              2
LATITUD               2
PARTICIPANTES         2
VICTIMA               2
ACUSADO               2
dtype: int64

In [36]:
# veamos el indice de los registros donde existen esos valores nulos, y a su vez transformamos los datos de la columna 'HH' a tipo int

indices = [] # declaramos la variable donde almacenaremos los indices que contengan datos nulos

for idx,x in enumerate(df_unido['HORA_HECHO']):
    try:
        if x == 'SD':
            continue
        else:
            df_unido['HORA_HECHO'][idx] = int(x)
    except ValueError as e:
        indices.append(idx)


C:\Users\usuario\AppData\Local\Temp\ipykernel_12528\3600952415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unido['HORA_HECHO'][idx] = int(x)


In [37]:
print(indices)

[65, 204]


In [38]:
registros_nulos = df_unido.iloc[indices]
registros_nulos
# aqui vemos los registros donde existen datos nulos, luego del merge entre dataframes (se colo un registro donde no existia dato que pueda transformarse a tipo 
# integer)

,ID_hecho,FECHA,ROL,VEHICULO_FALLECIDO,SEXO,EDAD,N_VICTIMAS,HORA_HECHO,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO
65,2016-0087,2016-07-03,CONDUCTOR,MOTO,MASCULINO,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,2017-0069,2017-05-29,CONDUCTOR,MOTO,MASCULINO,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# comprobemos que estos registros existan en ambos datasets

df_homicidios[df_homicidios['FECHA'].isin(registros_nulos['ID_hecho'])]

,ID_hecho,N_VICTIMAS,FECHA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO


In [40]:
df_homicidios[df_homicidios['FECHA'] == '2017-05-29']

,ID_hecho,N_VICTIMAS,FECHA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO


In [41]:
df_victimas[df_victimas['FECHA'] == '2017-05-29']

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
204,2017-0069,2017-05-29,2017,5,29,CONDUCTOR,MOTO,MASCULINO,39,2017-05-29 00:00:00


In [42]:
# en conclusion, los registros que contienen datos nulos luego del merge realizado, estan presentes en el dataframe df_victimas,
# no asi en el dataframe df_homicidios.... esto quiere decir que sea han registrado victimas fatales en el dataset VICTIMAS, pero no existe coincidencia 
# con registros realizados de accidentes o sinistros registrados en el dataset HECHOS.

In [43]:
df_unido.dtypes

ID_hecho                      object
FECHA                 datetime64[ns]
ROL                           object
VEHICULO_FALLECIDO            object
SEXO                          object
EDAD                          object
N_VICTIMAS                   float64
HORA_HECHO                    object
LUGAR_DEL_HECHO               object
TIPO_DE_CALLE                 object
COMUNA                       float64
LONGITUD                      object
LATITUD                       object
PARTICIPANTES                 object
VICTIMA                       object
ACUSADO                       object
dtype: object

In [44]:
# Veamos el caso de los datos en las columnas LATITUD y LONGITUD

In [45]:
indices = [] # declaramos la variable donde almacenaremos los indices que no contengan longitud

for idx,x in enumerate(df_unido['LONGITUD']):
    try:
        df_unido['LONGITUD'][idx] = float(x)
    except ValueError as e:
        indices.append(idx)

C:\Users\usuario\AppData\Local\Temp\ipykernel_12528\2345928273.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unido['LONGITUD'][idx] = float(x)


In [46]:
df_unido.iloc[indices]

,ID_hecho,FECHA,ROL,VEHICULO_FALLECIDO,SEXO,EDAD,N_VICTIMAS,HORA_HECHO,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO
39,2016-0052,2016-04-20,SD,MOTO,SD,SD,1.0,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,13.0,.,.,MOTO-SD,MOTO,SD
108,2016-0136,2016-10-25,CONDUCTOR,MOTO,SD,SD,1.0,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,4.0,.,.,MOTO-CARGAS,MOTO,CARGAS
121,2016-0151,2016-11-18,PEATON,PEATON,SD,SD,1.0,20,SD,CALLE,0.0,.,.,PEATON-SD,PEATON,SD
141,2016-0174,2016-12-27,SD,SD,SD,SD,1.0,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,0.0,.,.,SD-SD,SD,SD
182,2017-0042,2017-04-10,CONDUCTOR,MOTO,MASCULINO,SD,1.0,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,14.0,.,.,MOTO-CARGAS,MOTO,CARGAS
186,2017-0050,2017-04-28,CONDUCTOR,MOTO,MASCULINO,46,2.0,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,9.0,.,.,MOTO-CARGAS,MOTO,CARGAS
187,2017-0050,2017-04-28,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,16,2.0,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,9.0,.,.,MOTO-CARGAS,MOTO,CARGAS
188,2017-0051,2017-05-01,CONDUCTOR,AUTO,MASCULINO,33,1.0,3,AU DELLEPIANE 2400,AUTOPISTA,7.0,.,.,AUTO-AUTO,AUTO,AUTO
266,2017-0140,2017-11-19,CONDUCTOR,MOTO,MASCULINO,24,1.0,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,4.0,.,.,MOTO-PASAJEROS,MOTO,PASAJEROS
327,2018-0039,2018-04-21,PEATON,PEATON,MASCULINO,37,1.0,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,14.0,.,.,PEATON-AUTO,PEATON,AUTO


In [47]:
# Definimos una lista de tuplas con las geocordenadas de los registros faltantes, obtenidos desde Google Maps

datos = [(-34.5491067,-58.4393443),(-34.7609586,-58.1853702), (0,0),(-34.6281032,-58.4303422), (-34.5483931,-58.4416617),
         (-34.6411805,-58.5332558),(-34.6411805,-58.5332558),(-34.6692181,-58.4792622),(-34.6376995,-58.3793597),
         (-34.5511969,-58.4395778), (-34.5495252,-58.439771), (-34.6633327,-58.4978465), (-34.647772,-58.3592012)]

In [48]:
# Llenaremos los casilleros faltantes de datos de latitud y longitud con datos de geolocalizacion plana recolectados, de manera manual

for idx,dato in zip(indices,datos):
    df_unido['LATITUD'].iloc[idx] = float(dato[0])
    df_unido['LONGITUD'].iloc[idx] = float(dato[1])


C:\Users\usuario\AppData\Local\Temp\ipykernel_12528\1934034391.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unido['LATITUD'].iloc[idx] = float(dato[0])
C:\Users\usuario\AppData\Local\Temp\ipykernel_12528\1934034391.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unido['LONGITUD'].iloc[idx] = float(dato[1])


In [49]:
# vemos los valores de VEHICULO_FALLECIDO

df_unido['VICTIMA'].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', nan, 'CARGAS', 'BICICLETA',
       'PASAJEROS', 'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO'], dtype=object)

In [50]:
# vemos los valores de VEHICULO_FALLECIDO
df_unido['ACUSADO'].unique()

array(['AUTO', 'PASAJEROS', 'SD', 'OBJETO FIJO', 'CARGAS', 'MOTO',
       'MULTIPLE', 'OTRO', nan, 'BICICLETA', 'TREN'], dtype=object)

In [51]:
# Reemplazamos los valores de 'PEATON_MOTO' de la columna 'VICTIMA' ya que no aparece en el diccionario de datos, y los agrupamos en un valor denominado 'OTROS'
# que si se encuentra registrado en nuestro diccionario de datos

df_unido['VICTIMA'] = df_unido['VICTIMA'].replace('PEATON_MOTO', 'OTROS')

In [52]:
# Reemplazamos los valores 'SD' que no nos permiten convertir la columna a tipo int por valores estimados de la media y la moda respectivamente para las columnas 'EDAD' y 'HORA_HECHO'
df_unido['EDAD'] = df_unido['EDAD'].replace('SD', np.nan)
df_unido['EDAD'] = df_unido['EDAD'].fillna(round(df_unido['EDAD'].mean()))
df_unido['HORA_HECHO'] = df_unido['HORA_HECHO'].fillna(df_unido['HORA_HECHO'].mode().iloc[0]).replace('SD', df_unido['HORA_HECHO'].mode().iloc[0])

# Reemplazamos valores nan por un valor de referencia para formatear los valores de los datos a tipo int y luego los reconvertimos a str, para tratarlos como etiqueta
# en la columna 'COMUNA'

df_unido['COMUNA'] = df_unido['COMUNA'].fillna('-1')
df_unido['COMUNA'] = df_unido['COMUNA'].astype(int).astype(str)
df_unido['COMUNA'] = df_unido['COMUNA'].replace('-1', 'SD')

# Reemplazamos valores nan de la columna 'N_VICTIMAS' por el valor de la media de victimas del dataframe

df_unido['N_VICTIMAS'] = df_unido['N_VICTIMAS'].fillna(round(df_unido['N_VICTIMAS'].mean()))

# Reemplazamos los valores 'SD' de la columna SEXO por el valor modal

df_unido['SEXO'] = df_unido['SEXO'].replace('SD', df_unido['SEXO'].mode().iloc[0])


In [53]:
# Cambiamos el tipo de dato para las columnas a continuacion, para un mejor manejo de los datos

df_unido['HORA_HECHO'] = df_unido['HORA_HECHO'].astype(int)
df_unido['N_VICTIMAS'] = df_unido['N_VICTIMAS'].astype(int)
df_unido['LONGITUD'] = df_unido['LONGITUD'].astype(float)
df_unido['LATITUD'] = df_unido['LATITUD'].astype(float)
df_unido['EDAD'] = df_unido['EDAD'].astype(int)

In [54]:
df_unido.dtypes

ID_hecho                      object
FECHA                 datetime64[ns]
ROL                           object
VEHICULO_FALLECIDO            object
SEXO                          object
EDAD                           int32
N_VICTIMAS                     int32
HORA_HECHO                     int32
LUGAR_DEL_HECHO               object
TIPO_DE_CALLE                 object
COMUNA                        object
LONGITUD                     float64
LATITUD                      float64
PARTICIPANTES                 object
VICTIMA                       object
ACUSADO                       object
dtype: object

In [55]:
# reemplazamos los valores nulos de las columnas numericas 'LATITUD' y 'LONGITUD'
df_unido['LATITUD'] = df_unido['LATITUD'].fillna(0.0)
df_unido['LONGITUD'] = df_unido['LONGITUD'].fillna(0.0)

# Reemplazamos los valores nulos de todas las demas columnas object por el valor 'SD'
df_unido = df_unido.fillna('SD')

In [56]:
df_unido.isna().sum()

ID_hecho              0
FECHA                 0
ROL                   0
VEHICULO_FALLECIDO    0
SEXO                  0
EDAD                  0
N_VICTIMAS            0
HORA_HECHO            0
LUGAR_DEL_HECHO       0
TIPO_DE_CALLE         0
COMUNA                0
LONGITUD              0
LATITUD               0
PARTICIPANTES         0
VICTIMA               0
ACUSADO               0
dtype: int64

In [57]:
df_unido

,ID_hecho,FECHA,ROL,VEHICULO_FALLECIDO,SEXO,EDAD,N_VICTIMAS,HORA_HECHO,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,1,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO
1,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70,1,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30,1,7,AV ENTRE RIOS 2034,AVENIDA,1,-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO
3,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18,1,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,8,-58.465039,-34.680930,MOTO-SD,MOTO,SD
4,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29,1,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,1,-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,PEATON,PEATON,FEMENINO,50,1,6,AV. RIVADAVIA Y AV. PUEYRREDON,AVENIDA,3,-58.405969,-34.610120,PEATON-AUTO,PEATON,AUTO
713,2021-0093,2021-12-13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,1,17,AV. RIESTRA Y MOM,AVENIDA,7,-58.433538,-34.645616,MOTO-AUTO,MOTO,AUTO
714,2021-0094,2021-12-20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,1,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,9,-58.467398,-34.651178,MOTO-AUTO,MOTO,AUTO
715,2021-0095,2021-12-30,CONDUCTOR,MOTO,MASCULINO,27,1,0,AV. GAONA Y TERRADA,AVENIDA,11,-58.472934,-34.619847,MOTO-CARGAS,MOTO,CARGAS


In [58]:
# guardamos el dataframe en formato .csv

df_unido.to_csv('../data/df_unido.csv', encoding= 'utf-8')

In [59]:
# Guardamos el archivo en formato .parquet

# pq.write_table(pa.Table.from_pandas(df_unido), '../data/df_unido.parquet')